## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-26-18-50-27 +0000,nypost,LAX braces for another bumpy travel day as pro...,https://nypost.com/2025/11/26/us-news/lax-prot...
1,2025-11-26-18-50-00 +0000,wsj,U.S. Natural Gas Storage Down for Second Strai...,https://www.wsj.com/finance/commodities-future...
2,2025-11-26-18-49-55 +0000,nyt,Live Updates: Fire Engulfs Hong Kong Apartment...,https://www.nytimes.com/live/2025/11/26/world/...
3,2025-11-26-18-47-59 +0000,startribune,Conflicting accounts emerge after St. Paul ICE...,https://www.startribune.com/conflicting-accoun...
4,2025-11-26-18-46-21 +0000,bbc,Salary sacrifice tax break cut on workplace pe...,https://www.bbc.com/news/articles/cd9zx8z5d1no...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2314/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
63,trump,50
149,year,21
225,new,19
335,ukraine,16
65,will,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
156,2025-11-26-11-52-06 +0000,nypost,Foreigners will have to pay extra $100 to ente...,https://nypost.com/2025/11/26/us-news/trump-ad...,128
253,2025-11-26-01-40-51 +0000,nypost,Top Trump aide Steve Witkoff coached Russians ...,https://nypost.com/2025/11/25/world-news/top-t...,121
89,2025-11-26-16-23-23 +0000,cbc,New prosecutor won't pursue charges against Tr...,https://www.cbc.ca/news/world/trump-georgia-el...,119
353,2025-11-25-20-03-41 +0000,nypost,Trump dispatches top envoys to Russia and Ukra...,https://nypost.com/2025/11/25/us-news/trump-sa...,115
94,2025-11-26-16-01-38 +0000,nypost,Georgia drops 2020 election case against Trump...,https://nypost.com/2025/11/26/us-news/peter-sk...,108


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
156,128,2025-11-26-11-52-06 +0000,nypost,Foreigners will have to pay extra $100 to ente...,https://nypost.com/2025/11/26/us-news/trump-ad...
253,71,2025-11-26-01-40-51 +0000,nypost,Top Trump aide Steve Witkoff coached Russians ...,https://nypost.com/2025/11/25/world-news/top-t...
89,69,2025-11-26-16-23-23 +0000,cbc,New prosecutor won't pursue charges against Tr...,https://www.cbc.ca/news/world/trump-georgia-el...
15,63,2025-11-26-18-29-46 +0000,wapo,"Fire engulfs high-rise buildings in Hong Kong,...",https://www.washingtonpost.com/world/2025/11/2...
331,52,2025-11-25-21-14-00 +0000,wsj,At the White House’s annual—and typically stai...,https://www.wsj.com/politics/policy/trump-pard...
162,47,2025-11-26-11-13-21 +0000,nypost,Marty Makary touts FDA efforts to crack down o...,https://nypost.com/2025/11/26/us-news/marty-ma...
354,37,2025-11-25-20-01-00 +0000,wsj,"AI Startup Suno Inks Warner Music Deal, Settli...",https://www.wsj.com/business/media/ai-startup-...
149,35,2025-11-26-12-10-59 +0000,wapo,Two hostages’ bodies remain in Gaza as another...,https://www.washingtonpost.com/world/2025/10/1...
152,33,2025-11-26-12-00-00 +0000,startribune,"In Minnesota farm country, a plan for a $4 bil...",https://www.startribune.com/in-minnesota-farm-...
180,33,2025-11-26-10-54-39 +0000,bbc,Bolsonaro ordered to start serving 27-year pri...,https://www.bbc.com/news/articles/cr4dl19npv5o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
